In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
# 현재경로 확인
os.getcwd()

'C:\\실습자료\\part4'

In [15]:
# 데이터 불러오기
data = pd.read_csv("./otto_train.csv") # Product Category
data.head() # 데이터 확인

,id,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93,target
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,Class_1
1,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,3,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Class_1
3,4,1,0,0,1,6,1,5,0,0,...,0,1,2,0,0,0,0,0,0,Class_1
4,5,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Class_1


In [3]:
'''
id: 고유 아이디
feat_1 ~ feat_93: 설명변수
target: 타겟변수 (1~9)
'''

'\nid: 고유 아이디\nfeat_1 ~ feat_93: 설명변수\ntarget: 타겟변수 (1~9)\n'

In [16]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 61878 nVar: 95


## 의미가 없다고 판단되는 변수 제거

In [17]:
data = data.drop(['id'], axis = 1) # id 제거

## 타겟 변수의 문자열을 숫자로 변환

In [18]:
mapping_dict = {"Class_1": 1,
                "Class_2": 2,
                "Class_3": 3,
                "Class_4": 4,
                "Class_5": 5,
                "Class_6": 6,
                "Class_7": 7,
                "Class_8": 8,
                "Class_9": 9}
after_mapping_target = data['target'].apply(lambda x: mapping_dict[x])

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [19]:
feature_columns = list(data.columns.difference(['target'])) # target을 제외한 모든 행
X = data[feature_columns] # 설명변수
y = after_mapping_target # 타겟변수
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할| 
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(49502, 93) (12376, 93) (49502,) (12376,)


## 1. XGBoost

In [20]:
# !pip install xgboost
import xgboost as xgb
import time
start = time.time() # 시작 시간 지정
xgb_dtrain = xgb.DMatrix(data = train_x, label = train_y) # 학습 데이터를 XGBoost 모델에 맞게 변환
xgb_dtest = xgb.DMatrix(data = test_x) # 평가 데이터를 XGBoost 모델에 맞게 변환
xgb_param = {'max_depth': 10, # 트리 깊이
         'learning_rate': 0.01, # Step Size
         'n_estimators': 100, # Number of trees, 트리 생성 개수
         'objective': 'multi:softmax', # 목적 함수
        'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
xgb_model = xgb.train(params = xgb_param, dtrain = xgb_dtrain) # 학습 진행
xgb_model_predict = xgb_model.predict(xgb_dtest) # 평가 데이터 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, xgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

[23:11:47] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.0.0\src\learner.cc:328: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Accuracy: 76.67 %
Time: 13.32 seconds


In [23]:
xgb_model_predict

array([5., 3., 6., ..., 9., 2., 7.], dtype=float32)

## 2. LightGBM

In [21]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'objective': 'multiclass', # 목적 함수
            'num_class': len(set(train_y)) + 1} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
lgb_model_predict = np.argmax(lgb_model.predict(test_x), axis = 1) # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측
print("Accuracy: %.2f" % (accuracy_score(test_y, lgb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

Accuracy: 73.57 %
Time: 10.38 seconds


In [24]:
lgb_model.predict(test_x)

array([[9.14195396e-16, 2.27101123e-02, 3.85432853e-01, ...,
        3.25763637e-02, 7.61804989e-02, 4.64683926e-02],
       [1.16264426e-15, 3.77262272e-02, 2.26739830e-01, ...,
        1.92812290e-01, 1.01826669e-01, 8.11988908e-02],
       [8.03061518e-16, 1.74144816e-02, 1.18616633e-01, ...,
        2.57008295e-02, 6.69196402e-02, 4.08194769e-02],
       ...,
       [8.35633463e-16, 5.03899675e-02, 1.67414994e-01, ...,
        4.81925245e-02, 1.02188157e-01, 3.95359380e-01],
       [1.01560525e-15, 2.13472780e-02, 4.60509516e-01, ...,
        3.32864538e-02, 8.46310485e-02, 5.16230380e-02],
       [8.92775603e-16, 2.04756016e-02, 1.31867900e-01, ...,
        4.13697353e-01, 1.44381292e-01, 4.53796283e-02]])

## 3. Catboost

In [22]:
# !pip install catboost
import catboost as cb
start = time.time() # 시작 시간 지정
cb_dtrain = cb.Pool(data = train_x, label = train_y) # 학습 데이터를 Catboost 모델에 맞게 변환
cb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 100, # Number of trees, 트리 생성 개수
            'eval_metric': 'Accuracy', # 평가 척도
            'loss_function': 'MultiClass'} # 손실 함수, 목적 함수
cb_model = cb.train(pool = cb_dtrain, params = cb_param) # 학습 진행
cb_model_predict = np.argmax(cb_model.predict(test_x), axis = 1) + 1 # 평가 데이터 예측, Softmax의 결과값 중 가장 큰 값의 Label로 예측, 인덱스의 순서를 맞추기 위해 +1
print("Accuracy: %.2f" % (accuracy_score(test_y, cb_model_predict) * 100), "%") # 정확도 % 계산
print("Time: %.2f" % (time.time() - start), "seconds") # 코드 실행 시간 계산

0:	learn: 0.5907034	total: 844ms	remaining: 1m 23s
1:	learn: 0.6356107	total: 1.61s	remaining: 1m 18s
2:	learn: 0.6411256	total: 2.37s	remaining: 1m 16s
3:	learn: 0.6480344	total: 3.16s	remaining: 1m 15s
4:	learn: 0.6508222	total: 4.08s	remaining: 1m 17s
5:	learn: 0.6499939	total: 5.18s	remaining: 1m 21s
6:	learn: 0.6507818	total: 6.18s	remaining: 1m 22s
7:	learn: 0.6548422	total: 7.11s	remaining: 1m 21s
8:	learn: 0.6559533	total: 8.11s	remaining: 1m 21s
9:	learn: 0.6560947	total: 8.93s	remaining: 1m 20s
10:	learn: 0.6568421	total: 9.72s	remaining: 1m 18s
11:	learn: 0.6588219	total: 10.5s	remaining: 1m 17s
12:	learn: 0.6592259	total: 11.4s	remaining: 1m 16s
13:	learn: 0.6611248	total: 12.2s	remaining: 1m 15s
14:	learn: 0.6625591	total: 13s	remaining: 1m 13s
15:	learn: 0.6631853	total: 13.8s	remaining: 1m 12s
16:	learn: 0.6639328	total: 14.7s	remaining: 1m 11s
17:	learn: 0.6668821	total: 15.9s	remaining: 1m 12s
18:	learn: 0.6669630	total: 16.8s	remaining: 1m 11s
19:	learn: 0.6675286	tot

In [25]:
cb_model.predict(test_x)

array([[-0.35426047,  1.22109587,  0.44230101, ..., -0.1698448 ,
        -0.02059177, -0.2130643 ],
       [-0.07235138,  0.42535181,  0.20060428, ...,  0.21863604,
         0.2719157 ,  0.25089315],
       [-0.3315885 , -0.31862353, -0.31279765, ..., -0.29798357,
        -0.24018767, -0.32984969],
       ...,
       [ 0.05304325,  0.02500267, -0.14752573, ..., -0.20741963,
         0.12789417,  1.51166757],
       [-0.55093666,  1.7691278 ,  0.99746884, ..., -0.3420542 ,
        -0.49799871, -0.38136323],
       [-0.3033724 ,  0.09352675, -0.11808658, ...,  0.65825036,
         1.05515787, -0.20799899]])

In [27]:
# 데이터 불러오기
data = pd.read_csv("./kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [28]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

In [29]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


In [39]:
# !pip install lightgbm
import lightgbm as lgb
start = time.time() # 시작 시간 지정
lgb_dtrain = lgb.Dataset(data = train_x, label = train_y) # 학습 데이터를 LightGBM 모델에 맞게 변환
lgb_param = {'max_depth': 10, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행


In [40]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sqrt(mean_squared_error(lgb_model.predict(test_x),test_y))

210904.17249451784

## Ensemble의 Ensemble

In [46]:
train_x.iloc[[1,2,3],[0,1]]

,bathrooms,bedrooms
12412,1.75,4
7691,2.25,4
12460,1.75,2


In [56]:
import random
bagging_predict_result = [] # 빈 리스트 생성
for _ in range(30):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    #variable_index=[variable_index for variable_index in range(train_x.shape[1])]
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    #random_variable_index=np.random.choice(variable_index,train_x.shape[1])
    #print(random_variable_index)
    print('cal: ',len(set(random_data_index)))
    lgb_dtrain = lgb.Dataset(data = train_x.iloc[random_data_index], label = train_y.iloc[random_data_index]) # 학습 데이터를 LightGBM 모델에 맞게 변환
    lgb_param = {'max_depth': 14, # 트리 깊이
            'learning_rate': 0.01, # Step Size
            'n_estimators': 500, # Number of trees, 트리 생성 개수
            'objective': 'regression'} # 파라미터 추가, Label must be in [0, num_class) -> num_class보다 1 커야한다.
    lgb_model = lgb.train(params = lgb_param, train_set = lgb_dtrain) # 학습 진행
    predict1 = lgb_model.predict(test_x) # 테스트 데이터 예측
    print(sqrt(mean_squared_error(predict1,test_y)))
    bagging_predict_result.append(predict1) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장

cal:  9569


C:\Users\jhyun\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


218822.1037829121
cal:  9542


C:\Users\jhyun\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


213280.1520981975
cal:  9615


...

211582.34665707665
cal:  9606


C:\Users\jhyun\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


215995.5845275486
cal:  9551


C:\Users\jhyun\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


215206.25165399018


In [51]:
bagging_predict_result

[array([572407.77964023, 605870.82612042, 572407.77964023, ...,
        572407.77964023, 572407.77964023, 572407.77964023]),
 array([1012747.30841735, 1012747.30841735, 1012747.30841735, ...,
        1012747.30841735, 1012747.30841735, 1012747.30841735]),
 array([426894.27153769, 416329.90127117, 426894.27153769, ...,
        416329.90127117, 426894.27153769, 426894.27153769]),
 array([2495156.89640838, 2495156.89640838, 2495156.89640838, ...,
        2495156.89640838, 2495156.89640838, 2495156.89640838]),
 array([1258836.43387741, 2101856.14034692,  711822.98449465, ...,
         516499.69711713,  557974.99519599,  702736.85154025]),
 array([225651.6395935 , 219025.50772265, 225651.6395935 , ...,
        237158.80488776, 257672.09715365, 225651.6395935 ]),
 array([183431.86795099, 302640.11424258, 183431.86795099, ...,
        232173.90870368, 183613.45871361, 183431.86795099]),
 array([418468.46528199, 398474.34155067, 410625.41817993, ...,
        370229.67922729, 386335.8744828 , 4

In [57]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [58]:
# 예측한 결과값들의 평균을 계산하여 실제 테스트 데이트의 타겟변수와 비교하여 성능 평가

print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict, test_y)))) # RMSE

RMSE: 210144.9502511261


In [62]:
from sklearn import svm
model=svm.SVR()
svm_model=model.fit(train_x,train_y)
predict=svm_model.predict(test_x)

C:\Users\jhyun\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [64]:
print("RMSE: {}".format(sqrt(mean_squared_error(predict, test_y)))) # RMSE

RMSE: 392051.61716257443


In [83]:
from sklearn.neural_network import MLPRegressor
model2=MLPRegressor(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(7,5),random_state=1)
nn_model=model2.fit(train_x,train_y)
predict2=nn_model.predict(test_x)
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE

RMSE: 258938.38859403002


In [84]:
bagging_predict_result = [] # 빈 리스트 생성
for _ in range(30):
    data_index = [data_index for data_index in range(train_x.shape[0])] # 학습 데이터의 인덱스를 리스트로 변환
    #variable_index=[variable_index for variable_index in range(train_x.shape[1])]
    random_data_index = np.random.choice(data_index, train_x.shape[0]) # 데이터의 1/10 크기만큼 랜덤 샘플링, // 는 소수점을 무시하기 위함
    #random_variable_index=np.random.choice(variable_index,train_x.shape[1])
    #print(random_variable_index)
    
    print('cal: ',len(set(random_data_index)))
    model2=MLPRegressor(solver='lbfgs',alpha=1e-5,hidden_layer_sizes=(7,5),random_state=1)
    nn_model=model2.fit(train_x.iloc[random_data_index],train_y.iloc[random_data_index])
    predict2=nn_model.predict(test_x)
    print(sqrt(mean_squared_error(predict2,test_y)))
    bagging_predict_result.append(predict2) # 반복문이 실행되기 전 빈 리스트에 결과 값 저장

cal:  9527
252484.938210846
cal:  9591
262802.5092084049
cal:  9527
260548.4763743648
cal:  9583
262826.84790043405
cal:  9485
262165.4210504962
cal:  9559
261909.9081755877
cal:  9561
255942.14100767413
cal:  9554
263496.71964314976
cal:  9546
261660.94044744535
cal:  9531
263050.02582904196
cal:  9544
255989.35909405255
cal:  9542
263372.37117729936
cal:  9505
262380.33262930205
cal:  9581
264351.2861078123
cal:  9526
263636.76715713926
cal:  9612
263653.39792088326
cal:  9608
272915.4331693652
cal:  9610
258319.5641196897
cal:  9545
260863.03802063217
cal:  9572
253831.52611060187
cal:  9575
263896.56206299935
cal:  9546
263885.353337453
cal:  9527
264530.949857093
cal:  9600
263367.86590101867
cal:  9570
262998.33020048845
cal:  9506
264654.66353501123
cal:  9544
263474.21190882847
cal:  9562
263787.31055612664
cal:  9583
262714.9210382397
cal:  9584
263511.97235680424


In [85]:
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict2 = [] # 빈 리스트 생성
for lst2_index in range(test_x.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict2.append(np.mean(temp_predict)) # 해당 인덱스의 30개의 결과값에 대한 평균을 최종 리스트에 추가

In [86]:
print("RMSE: {}".format(sqrt(mean_squared_error(bagging_predict2, test_y)))) # RMSE

RMSE: 260756.59443881395
